## 1.数据预处理

Data Preprocessing

In [1]:
# 控制调试输出的开关
DEBUG = not False

### 1.1 配置参数 
> 需根据实际文件路径修改
- 需要解析 PDF 文件路径
- 输出文件 CSV 路径
- 输出文件 CSV 表头

In [2]:
CONST_PDF_PATH = "./assets/2024年四川省重点项目名单.pdf"  
CONST_OUTPUT_CSV = "./outputs/2024年四川重点项目解析结果.csv"
CONST_ORIGIN_HEADERS = ["总序号", "分序号", "项目名称", "建设地址"]
CONST_TARGET_HEADERS = ["项目名称", "建设地址", "一级项目领域", "二级项目领域", "项目类型"]


### 1.2 使用 pdfplumber 解析 PDF 页面数据

In [15]:
import pdfplumber

with pdfplumber.open(CONST_PDF_PATH) as pdf:
    page = pdf.pages[0]
    img = page.crop(bbox=[0, 0, 584, 142]).to_image()  # 生成PIL图像对象
    img.show()  # 显示图像，手动观察并估算坐标

In [16]:
import pdfplumber
import re

# 申明一个(上下文)变量，一级项目领域，类型为字符串
Level_1_Project_Domain = ""
Level_2_Project_Domain = ""
Level_3_Project_Domain = ""

def extract_table_from_pdf(pdf_path):
    result_key_project_list = []  # 存储所有页面的表格数据

    with pdfplumber.open(pdf_path) as pdf:
        # 获取实际总页数
        total_pages = len(pdf.pages)  
        
        # 遍历PDF的每一页
        for page_num in range(0, total_pages):
        # for page_num in range(0, 10):
            page = pdf.pages[page_num]
            print(f"正在解析第 {page_num + 1} 页...")
            
            # 提取当前页所有表格
            table = page.extract_table(
                # 参考：
                # https://github.com/jsvine/pdfplumber?tab=readme-ov-file#table-extraction-settings
                table_settings={
                    "vertical_strategy": "lines",  # 按竖线识别列边界
                    "horizontal_strategy": "lines",  # 按横线识别行边界
                    "text_y_tolerance": 10,  # 文本垂直方向容错
                    "text_x_tolerance": 5,   # 文本水平方向容错
                }
            )

            # 提取页面标题，解析(项目类型)
            title_text = page.crop(bbox=[0, 0, 584, 142]).extract_text() or ""
            if DEBUG:
                print(f"🌟第 {page_num + 1} 页标题：{title_text}")
            
            # 使用新函数处理表格数据
            table_data = process_table_rows(table)

            if not DEBUG:
                print(f"🌟第 {page_num + 1} 页处理后的数据：")
                for row in table_data:
                    print(row)

            # 将处理后的数据添加到总数据列表中
            result_key_project_list.extend(table_data)
    
    return result_key_project_list

def process_table_rows(table):
    """
    处理单个表格的所有行数据
    
    Args:
        table: 单个表格的数据（二维列表）
        target_headers: 目标表头列表
        
    Returns:
        tuple: (处理后的表头计数, 该表格处理后的数据列表)
    """

    # 存储当前表格处理后的数据
    # example:
    # [
    #     ["项目名称", "建设地址", "一级项目领域", "二级项目领域", "项目类型"]
    #     [……], 
    # ]
    result_list = []  

    for index, row in enumerate(table):

        if not DEBUG:
            print(f"正在处理第 {index + 1} 行：{row}")

        # 跳过表头行
        if index < 2:
            continue  

        # 解构为单元格，总序号、分序号、项目名称、建设地址
        # 移除两侧潜在的空格
        total_seq_cell, sub_seq_cell, project_name_cell, construction_address_cell = [cell.strip() if cell else "" for cell in row]

        # 清理项目名称单元格，去除换行符和括号及其后内容
        # "教育、文化、体育及社会服务（16项）" -> "教育、文化、体育及社会服务"
        project_name_cell = remove_after_first_paren(project_name_cell.replace("\n", ""))

        global Level_1_Project_Domain 
        global Level_2_Project_Domain 
        global Level_3_Project_Domain 

        # total_seq_cell 为 '一二三四五' 其中一个汉字，则使用 project_name_cell 作为 一级项目领域
        if re.match(r'^[一二三四五]$', total_seq_cell):
            Level_1_Project_Domain = project_name_cell
            Level_2_Project_Domain = ""
            Level_3_Project_Domain = ""
            continue

        if re.match(r'^（[一二三四五]）$', total_seq_cell):
            Level_2_Project_Domain = project_name_cell
            Level_3_Project_Domain = ""
            continue

        if total_seq_cell in ['I', 'II', 'III', 'IV', 'V', 'VI']:
            Level_3_Project_Domain = project_name_cell
            continue

        # 跳过建设地址为空的行
        if not construction_address_cell or construction_address_cell == '':
            continue

        # 建设地址的换行符处理为空格
        address_city = construction_address_cell.replace("\n", " ")

        refined_data = [project_name_cell, address_city, Level_1_Project_Domain, Level_2_Project_Domain]

        if not DEBUG:
            print(f"🌈已经处理第 {index + 1} 行：{refined_data}")

        result_list.append(refined_data)

    return result_list

def remove_after_first_paren(text):
    # 使用正则表达式匹配第一个左括号及其后面的所有内容并替换为空字符串
    return re.sub(r'（.*', '', text)
    
# 提取表格数据
result_key_project_list = extract_table_from_pdf(CONST_PDF_PATH) 

if not DEBUG:
    print(f"🌟所有表格处理后的数据：")
    for row in result_key_project_list:
        print(row)
    

正在解析第 1 页...
🌟第 1 页标题：附件
2024年四川省重点项目名单（续建）
正在解析第 2 页...
🌟第 2 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 3 页...
🌟第 3 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 4 页...
🌟第 4 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 5 页...
🌟第 5 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 6 页...
🌟第 6 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 7 页...
🌟第 7 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 8 页...
🌟第 8 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 9 页...
🌟第 9 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 10 页...
🌟第 10 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 11 页...
🌟第 11 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 12 页...
🌟第 12 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 13 页...
🌟第 13 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 14 页...
🌟第 14 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 15 页...
🌟第 15 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 16 页...
🌟第 16 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 17 页...
🌟第 17 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 18 页...
🌟第 18 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 19 页...
🌟第 19 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 20 页...
🌟第 20 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 21 页...
🌟第 21 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 22 页...
🌟第 22 页标题：2024年四川省重点项目名单（续建）
序号
正在解析第 

### 1.3 使用 pandas 数据清洗

转换为 DataFrame 并清洗

In [ ]:
import pandas as pd

def convert_to_dataframe(all_tables_data):
    # 将列表转为 DataFrame，用目标表头命名列
    df = pd.DataFrame(all_tables_data[1:], columns=all_tables_data[0])  # 0行是表头，1行后是数据

    # 进一步清洗：删除全空行、处理缺失值
    df = df.dropna(how="all")  # 删除全空行

    # 确保列存在再进行填充操作
    if "总序号" in df.columns:
        df["总序号"] = df["总序号"].fillna("")  # 总序号为空的填充为空字符串（如子项目）
    if "分序号" in df.columns:
        df["分序号"] = df["分序号"].fillna("")  # 分序号为空的填充为空字符串

    # 添加“项目类型”列
    if "总序号" in df.columns:
        df["项目类型"] = df["总序号"].apply(lambda x: "续建" if (str(x).isdigit() and int(x) <= 504) else "新开工")
    else:
        print("警告：未找到'总序号'列，跳过'项目类型'生成。")
        df["项目类型"] = ""
    
    return df

### 1.4 保存结果（CSV）

In [ ]:
def save_results(df, output_csv):
    # 保存为 CSV（兼容更多工具，如 Excel、Python读取）
    # utf-8-sig 解决中文乱码
    df.to_csv(output_csv, index=False, encoding="utf-8-sig") 
    return output_csv


### 数据处理主函数运行 (main)
> 组织各个步骤函数运行

In [ ]:
from datetime import datetime

def mainDataPreprocess():
    # 配置参数
    pdf_path, output_csv, target_headers = prepare_params()
    
    # 提取表格数据
    all_tables_data = extract_table_from_pdf(pdf_path, target_headers)
    
    # 转换为DataFrame
    df = convert_to_dataframe(all_tables_data)
    
    # 保存结果
    saved_path = save_results(df, output_csv)

    # 获取当前时间
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # 输出结果信息
    print(f"解析完成({current_time})！共提取{len(df)}条项目数据，已保存至：")
    print(f"- CSV: {saved_path}")
    
    # 打印前5条数据验证结果
    print("\n解析结果预览（前5条）：")
    print(df.head())

# 运行主程序
if __name__ == "__main__":
    mainDataPreprocess()